In [1]:
import os

def getFilePathList(rootDir):
    filePath_list = []
    for walk in os.walk(rootDir):
        part_filePath_list = [os.path.join(walk[0], file) for file in walk[2]]
        filePath_list.extend(part_filePath_list)
    return filePath_list
filePath_list = getFilePathList('/home/ubuntu/project/tfTest/THUCNews/THUCNews')
len(filePath_list)

836075

In [2]:
label_list = []
for filePath in filePath_list:
    label = filePath.split('/')[-2]
    label_list.append(label)
len(label_list)

836075

In [3]:
import pickle

with open('label_list.pickle', 'wb') as file:
    pickle.dump(label_list, file)


In [15]:
import time
import pickle
import re

def getFile(filePath):
    with open(filePath, encoding='utf8') as file:
        fileStr = ''.join(file.readlines(1000))
    return fileStr

interval = 20000
n_samples = len(label_list)
startTime = time.time()
directory_name = 'content_list'
if not os.path.isdir(directory_name):
    os.mkdir(directory_name)
for i in range(0, n_samples, interval):
    startIndex = i
    endIndex = i + interval
    content_list = []
    print('%06d-%06d start' %(startIndex, endIndex))
    for filePath in filePath_list[startIndex:endIndex]:
        fileStr = getFile(filePath)
        content = re.sub('\s+', ' ', fileStr)
        content_list.append(content)
    save_fileName = directory_name + '/%06d-%06d.pickle' %(startIndex, endIndex)
    with open(save_fileName, 'wb') as file:
        pickle.dump(content_list, file)
    used_time = time.time() - startTime
    print('%06d-%06d used time: %.2f seconds' %(startIndex, endIndex, used_time))

000000-020000 start
000000-020000 used time: 1.47 seconds
020000-040000 start
020000-040000 used time: 3.03 seconds
040000-060000 start
040000-060000 used time: 4.55 seconds
060000-080000 start
060000-080000 used time: 5.91 seconds
080000-100000 start
080000-100000 used time: 7.35 seconds
100000-120000 start
100000-120000 used time: 8.71 seconds
120000-140000 start
120000-140000 used time: 10.02 seconds
140000-160000 start
140000-160000 used time: 11.61 seconds
160000-180000 start
160000-180000 used time: 13.33 seconds
180000-200000 start
180000-200000 used time: 14.78 seconds
200000-220000 start
200000-220000 used time: 16.17 seconds
220000-240000 start
220000-240000 used time: 17.51 seconds
240000-260000 start
240000-260000 used time: 18.90 seconds
260000-280000 start
260000-280000 used time: 20.25 seconds
280000-300000 start
280000-300000 used time: 21.61 seconds
300000-320000 start
300000-320000 used time: 23.04 seconds
320000-340000 start
320000-340000 used time: 24.45 seconds
340

In [4]:
import time
import pickle
import os

def getFilePathList(rootDir):
    filePath_list = []
    for walk in os.walk(rootDir):
        part_filePath_list = [os.path.join(walk[0], file) for file in walk[2]]
        filePath_list.extend(part_filePath_list)
    return filePath_list

startTime = time.time()
contentListPath_list = getFilePathList('content_list')
content_list = []
for filePath in contentListPath_list:
    with open(filePath, 'rb') as file:
        part_content_list = pickle.load(file)
    content_list.extend(part_content_list)
with open('label_list.pickle', 'rb') as file:
    label_list = pickle.load(file)
used_time = time.time() - startTime
print('used time: %.2f seconds' %used_time)
sample_size = len(content_list)
print('length of content_list，mean sample size: %d' %sample_size)

used time: 10.87 seconds
length of content_list，mean sample size: 836075


## 选取一部分数据

In [5]:
sample_number = len(label_list)

In [6]:
sample_number

836075

In [9]:
import random
select_index = random.sample(list(range(sample_number)), k=3000)

In [11]:
import numpy as np
label_list = np.array(label_list)[select_index]

In [12]:
len(label_list)

3000

In [14]:
len(content_list)

836075

In [15]:
content_list_new = []
for index in select_index:
    content_list_new.append(content_list[index])
content_list = content_list_new

## 词汇表

In [36]:
from collections import Counter 
def getVocabularyList(content_list, vocabulary_size):
    allContent_str = ''.join(content_list)
    counter = Counter(allContent_str)
    vocabulary_list = [k[0] for k in counter.most_common(vocabulary_size)]
    return ['PAD'] + vocabulary_list
startTime = time.time()
vocabulary_list = getVocabularyList(content_list, 5000)
used_time = time.time() - startTime
print('used time: %.2f seconds' %used_time)

used time: 0.64 seconds


In [ ]:
# sample_number = len(label——li)

## 数据准备

In [37]:
import time
startTime = time.time()
from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y = train_test_split(content_list, label_list)
train_content_list = train_X
train_label_list = train_y
test_content_list = test_X
test_label_list = test_y
used_time = time.time() - startTime
print('train_test_split used time : %.2f seconds' %used_time)
vocabulary_size = 5000  # 词汇表达小
sequence_length = 600  # 序列长度
embedding_size = 64  # 词向量维度
num_filters = 256  # 卷积核数目
filter_size = 5  # 卷积核尺寸
num_fc_units = 128  # 全连接层神经元
dropout_keep_probability = 0.5  # dropout保留比例
learning_rate = 1e-3  # 学习率
batch_size = 64  # 每批训练大小
word2id_dict = dict([(b, a) for a, b in enumerate(vocabulary_list)])
content2idList = lambda content : [word2id_dict[ word] for word in content if word in word2id_dict]
train_idlist_list = [content2idList(content) for content in train_content_list]
used_time = time.time() - startTime
print('content2idList used time : %.2f seconds' %used_time)
import numpy as np
num_classes = np.unique(label_list).shape[0]
import tensorflow.contrib.keras as kr
train_X = kr.preprocessing.sequence.pad_sequences(train_idlist_list, sequence_length)
from sklearn.preprocessing import LabelEncoder
labelEncoder = LabelEncoder()
train_y = labelEncoder.fit_transform(train_label_list)
train_Y = kr.utils.to_categorical(train_y, num_classes)
import tensorflow as tf
tf.reset_default_graph()
X_holder = tf.placeholder(tf.int32, [None, sequence_length])
Y_holder = tf.placeholder(tf.float32, [None, num_classes])
used_time = time.time() - startTime
print('data preparation used time : %.2f seconds' %used_time)

train_test_split used time : 0.01 seconds
content2idList used time : 0.60 seconds
data preparation used time : 0.78 seconds


## 查看数据

In [38]:
import pandas as pd
pd.value_counts(train_idlist_list[1])

1       23
2       17
4       16
122     14
9       13
128     10
36       9
3        9
14       9
575      8
1073     8
22       7
21       7
18       6
40       6
11       6
33       6
80       5
1141     5
783      5
165      5
196      5
339      5
107      5
7        5
992      5
26       5
2002     5
6        5
8        5
        ..
589      1
610      1
1185     1
638      1
539      1
151      1
1667     1
147      1
144      1
390      1
648      1
188      1
537      1
232      1
214      1
227      1
226      1
499      1
222      1
504      1
505      1
1530     1
535      1
203      1
510      1
519      1
2575     1
1212     1
187      1
279      1
Length: 244, dtype: int64

In [40]:
train_y[0]

12

In [ ]:
label_

## 搭建神经网络

In [9]:
#输入法好不习惯
#打个下划线贼难，变来变去的

tf.__version__

'1.10.0'

In [24]:
#你这版本太低了
#我的GPU只支持1.3-1.4

In [10]:
!nvidia-smi

Tue Nov  6 17:50:36 2018       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 384.130                Driver Version: 384.130                   |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX TIT...  Off  | 00000000:83:00.0  On |                  N/A |
| 32%   48C    P8    28W / 189W |    449MiB /  6075MiB |     14%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX TIT...  Off  | 00000000:84:00.0 Off |                  N/A |
| 29%   44C    P8    32W / 189W |      1MiB /  6082MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
        

In [20]:
import tensorflow as tf
import os
tf.reset_default_graph()
X_holder = tf.placeholder(tf.int32, [None, sequence_length])
Y_holder = tf.placeholder(tf.float32, [None, num_classes])
used_time = time.time() - startTime
print('data preparation used time : %.2f seconds' %used_time)

data preparation used time : 39.85 seconds


In [32]:
embedding = tf.get_variable('embedding', 
                            [vocabulary_size, embedding_size])
embedding_inputs = tf.nn.embedding_lookup(embedding,
                                          X_holder)
conv = tf.layers.conv1d(embedding_inputs,
                        num_filters,
                        filter_size)
max_pooling = tf.reduce_max(conv, 
                            [1])
full_connect = tf.layers.dense(max_pooling,
                               num_fc_units)
full_connect_dropout = tf.contrib.layers.dropout(full_connect, 
                                                 keep_prob=dropout_keep_probability)
full_connect_activate = tf.nn.relu(full_connect_dropout)
softmax_before = tf.layers.dense(full_connect_activate,
                                 num_classes)
predict_Y = tf.nn.softmax(softmax_before)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=Y_holder,
                                                           logits=softmax_before)
loss = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate)
train = optimizer.minimize(loss)
isCorrect = tf.equal(tf.argmax(Y_holder, 1), tf.argmax(predict_Y, 1))
accuracy = tf.reduce_mean(tf.cast(isCorrect, tf.float32))

## 参数初始化

In [33]:
init = tf.global_variables_initializer()
session = tf.Session()
session.run(init)

## 模型训练

In [35]:
test_idlist_list = [content2idList(content) for content in test_content_list]
test_X = kr.preprocessing.sequence.pad_sequences(test_idlist_list, sequence_length)
test_y = labelEncoder.transform(test_label_list)
test_Y = kr.utils.to_categorical(test_y, num_classes)
import random
for i in range(1000):
    selected_index = random.sample(list(range(len(train_y))), k=batch_size)
    batch_X = train_X[selected_index]
    batch_Y = train_Y[selected_index]
    session.run(train, {X_holder:batch_X, Y_holder:batch_Y})
    step = i + 1 
    if step % 100 == 0:
        selected_index = random.sample(list(range(len(test_y))), k=200)
        batch_X = test_X[selected_index]
        batch_Y = test_Y[selected_index]
        loss_value, accuracy_value = session.run([loss, accuracy], {X_holder:batch_X, Y_holder:batch_Y})
        print('step:%d loss:%.4f accuracy:%.4f' %(step, loss_value, accuracy_value))

step:100 loss:4.4178 accuracy:0.2900
step:200 loss:3.9606 accuracy:0.2600
step:300 loss:4.2849 accuracy:0.3250
step:400 loss:4.3718 accuracy:0.2750
step:500 loss:4.3940 accuracy:0.3150
step:600 loss:4.6178 accuracy:0.2350
step:700 loss:4.8709 accuracy:0.2500
step:800 loss:4.5706 accuracy:0.2600
step:900 loss:4.8859 accuracy:0.2800
step:1000 loss:5.1113 accuracy:0.2600


## 混淆矩阵

In [26]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix

def predictAll(test_X, batch_size=100):
    predict_value_list = []
    for i in range(0, len(test_X), batch_size):
        selected_X = test_X[i: i + batch_size]
        predict_value = session.run(predict_Y, {X_holder:selected_X})
        predict_value_list.extend(predict_value)
    return np.array(predict_value_list)

Y = predictAll(test_X)
y = np.argmax(Y, axis=1)
predict_label_list = labelEncoder.inverse_transform(y)
pd.DataFrame(confusion_matrix(test_label_list, predict_label_list), 
             columns=labelEncoder.classes_,
             index=labelEncoder.classes_ )

,体育,娱乐,家居,彩票,房产,教育,时尚,时政,星座,游戏,社会,科技,股票,财经
体育,41,14,0,3,5,1,0,5,0,2,10,11,26,1
娱乐,15,32,2,0,4,0,1,5,0,2,5,7,7,2
家居,2,1,2,0,0,0,0,4,0,2,1,5,1,0
彩票,0,0,0,0,0,0,0,0,0,0,1,1,0,0
房产,4,3,0,0,0,0,1,0,0,0,0,1,6,0
教育,6,0,1,0,0,2,2,5,0,0,1,5,9,0
时尚,1,0,0,0,0,2,0,0,0,0,1,4,4,0
时政,9,10,5,1,1,3,2,13,0,1,3,11,17,0
星座,2,0,0,0,0,0,1,0,0,0,1,0,3,0
游戏,2,4,0,0,0,0,0,1,0,3,0,13,1,2


## 报告表

In [28]:
import numpy as np 
from sklearn.metrics import precision_recall_fscore_support

def eval_model(y_true, y_pred, labels):
    # 计算每个分类的Precision, Recall, f1, support
    p, r, f1, s = precision_recall_fscore_support(y_true, y_pred)
    # 计算总体的平均Precision, Recall, f1, support
    tot_p = np.average(p, weights=s)
    tot_r = np.average(r, weights=s)
    tot_f1 = np.average(f1, weights=s)
    tot_s = np.sum(s)
    res1 = pd.DataFrame({
        u'Label': labels,
        u'Precision': p,
        u'Recall': r,
        u'F1': f1,
        u'Support': s
    })
    res2 = pd.DataFrame({
        u'Label': ['总体'],
        u'Precision': [tot_p],
        u'Recall': [tot_r],
        u'F1': [tot_f1],
        u'Support': [tot_s]
    })
    res2.index = [999]
    res = pd.concat([res1, res2])
    return res[['Label', 'Precision', 'Recall', 'F1', 'Support']]

eval_model(test_label_list, predict_label_list, labelEncoder.classes_)

,Label,Precision,Recall,F1,Support
0,体育,0.269737,0.344538,0.302583,119
1,娱乐,0.390244,0.390244,0.390244,82
2,家居,0.166667,0.111111,0.133333,18
3,彩票,0.000000,0.000000,0.000000,2
4,房产,0.000000,0.000000,0.000000,15
5,教育,0.076923,0.064516,0.070175,31
6,时尚,0.000000,0.000000,0.000000,12
7,时政,0.265306,0.171053,0.208000,76
8,星座,0.000000,0.000000,0.000000,7
9,游戏,0.176471,0.115385,0.139535,26


In [36]:
#可以看到房产，时尚，财经，预测准确率为0